Retrieve experiment results

In [3]:
import json
from os.path import join
import os

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedir_ids = [25, 26]
basedirs = [join(get_output_dir(), 'predict_multi', str(_id), 'run') for _id in basedir_ids]
res_list = []
for basedir in basedirs:
    for exp_dir in os.listdir(basedir):
        exp_dir = join(basedir, exp_dir)
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        alpha = config['alpha']
        beta = config['beta']
        seed = config['seed']
        model = config['model']
        source = config['source']
        score = info['score']
        rank = info['rank']
        normalized = config['with_std']
        res = {'datasets': datasets, 'alpha': alpha, 'beta': beta, 'seed': seed, 'rank': rank,
               'source': source, 'normalized': normalized,
              'model': model}
        for key, value in score.items():
            res[key] = value
        if basedir == join(get_output_dir(), 'predict_multi', '11', 'run') and model == 'logistic':
            continue
        res_list.append(res)
res = pd.DataFrame(res_list)

Compare results with trace norm, with and without transfer

In [4]:
df_agg = res.groupby(by=['normalized', 'source', 'model', 'datasets', 'alpha', 'beta']).aggregate(['mean', 'std'])

df_agg = df_agg.fillna(0)


dfs = {}
for dataset in ['brainomics', 'archi']:
    this_df = df_agg.query("datasets in ['%s__hcp', '%s']" % (dataset, dataset))
    test_mean = sum(this_df[('test_%s' % dataset, 'mean')] for dataset in [dataset, 'hcp']) / 2
    this_df = this_df.assign(test_mean=test_mean)
    indices = this_df[('test_%s' % dataset, 'mean')].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')
    this_df = this_df.loc[indices.values]
    dfs[dataset] = this_df

Compare results with 3 datasets

In [5]:
this_df = df_agg.query("datasets == 'brainomics__archi__hcp'")
test_mean = sum(this_df[('test_%s' % dataset, 'mean')] for dataset in ['brainomics', 'archi', 'hcp']) / 3
this_df = this_df.assign(test_mean=test_mean)
indices = this_df['test_mean'].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')
this_df = this_df.loc[indices.values]
dfs['archi+brainomics+hcp'] = this_df

In [6]:
result_df = pd.concat(dfs.values(), keys=dfs.keys(), names=['target'])

In [7]:
result_df

rank  \
                                                                                                         mean   
target               normalized source                 model    datasets               alpha    beta            
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  17.05   
                                                       trace    brainomics             0.003162 0.0000   9.50   
                                                                brainomics__hcp        0.003162 0.0000  19.05   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  29.05   
                                                       trace    archi                  0.001000 0.0000  18.85   
                                                                archi__hcp             0.000316 0.0000  38.05   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  52.00   

                                                                                                                  \
                                                                                                             std   
target               normalized source                 model    datasets               alpha    beta               
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  0.223607   
                                                       trace    brainomics             0.003162 0.0000  0.512989   
                                                                brainomics__hcp        0.003162 0.0000  0.825578   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  0.223607   
                                                       trace    archi                  0.001000 0.0000  0.670820   
                                                                archi__hcp             0.000316 0.0000  1.050063   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  1.000000   

                                                                                                              seed  \
                                                                                                              mean   
target               normalized source                 model    datasets               alpha    beta                 
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  2330081002   
                                                       trace    brainomics             0.003162 0.0000  2330081002   
                                                                brainomics__hcp        0.003162 0.0000  2330081002   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  2330081002   
                                                       trace    archi                  0.001000 0.0000  2330081002   
                                                                archi__hcp             0.000316 0.0000  2330081002   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  1243615929   

                                                                                                                      \
                                                                                                                 std   
target               normalized source                 model    datasets               alpha    beta                   
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  1.242420e+09   
                                                       trace    brainomics             0.003162 0.0000  1.242420e+09   
                                

In [10]:
df_agg.query("datasets == 'brainomics__archi__hcp'")

rank  \
                                                                                   mean   
normalized source                 model datasets               alpha    beta              
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   71.000000   
                                                               0.000001 0.0   71.000000   
                                                               0.000003 0.0   71.000000   
                                                               0.000010 0.0   71.000000   
                                                               0.000032 0.0   71.000000   
                                                               0.000100 0.0   70.666667   
                                                               0.000316 0.0   52.000000   
                                                               0.001000 0.0   28.000000   
                                                               0.003162 0.0   18.333333   
                                                               0.010000 0.0   10.000000   
                                                               0.031623 0.0    3.666667   
                                                               0.100000 0.0    1.000000   
                                                               0.316228 0.0    0.000000   
                                                               1.000000 0.0    0.000000   

                                                                                        \
                                                                                   std   
normalized source                 model datasets               alpha    beta             
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   0.000000   
                                                               0.000001 0.0   0.000000   
                                                               0.000003 0.0   0.000000   
                                                               0.000010 0.0   0.000000   
                                                               0.000032 0.0   0.000000   
                                                               0.000100 0.0   0.577350   
                                                               0.000316 0.0   1.000000   
                                                               0.001000 0.0   0.000000   
                                                               0.003162 0.0   1.154701   
                                                               0.010000 0.0   0.000000   
                                                               0.031623 0.0   0.577350   
                                                               0.100000 0.0   0.000000   
                                                               0.316228 0.0   0.000000   
                                                               1.000000 0.0   0.000000   

                                                                                    seed  \
                                                                                    mean   
normalized source                 model datasets               alpha    beta               
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   1243615929   
                                                               0.000001 0.0   1243615929   
                                                               0.000003 0.0   1243615929   
                                                               0.000010 0.0   1243615929   
                                                               0.000032 0.0   1243615929   
                                                               0.000100 0.0   1243615929   
                                                               0.000316 0.0   1243615929   
                                                               0.001000 0.0   124361